In [ ]:
#importaciones de modulos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from ipywidgets import interact, widgets, VBox, HBox, Output
from IPython.display import display, clear_output

In [ ]:

# --- 1. Carga y Estado del DataFrame ---

class InteractiveCleaner:
    def __init__(self, filepath, encoding='utf-8'): # Added encoding parameter with a default
        self.df_original = pd.read_csv(filepath, encoding=encoding) # Passed encoding to read_csv
        self.df_cleaned = self.df_original.copy()
        print(f"Dataset cargado: {self.df_cleaned.shape[0]} filas, {self.df_cleaned.shape[1]} columnas.")

    def show_missing(self):
        """Muestra el porcentaje de datos faltantes"""
        missing_percent = (self.df_cleaned.isnull().sum() / len(self.df_cleaned) * 100).sort_values(ascending=False)
        missing_percent = missing_percent[missing_percent > 0]
        if missing_percent.empty:
            print("¡Felicidades! No hay valores faltantes.")
            return None
        print("Valores Faltantes (%):")
        print(missing_percent)
        return missing_percent.index.tolist() # Devuelve columnas con faltantes

    def get_column_stats(self, col):
        """Muestra estadísticas rápidas para una columna"""
        print(f"--- Estadísticas de '{col}' ---")
        print(f"Tipo: {self.df_cleaned[col].dtype}")
        if pd.api.types.is_numeric_dtype(self.df_cleaned[col]):
            print(f"Media: {self.df_cleaned[col].mean():.2f}")
            print(f"Mediana: {self.df_cleaned[col].median():.2f}")
            print(f"StdDev: {self.df_cleaned[col].std():.2f}")
            #
        else:
            print(f"Valores únicos: {self.df_cleaned[col].nunique()}")
            print("Moda:", self.df_cleaned[col].mode().iloc[0])


    def run_interactive_cleaning(self):
        """El motor principal que genera los widgets"""

        # 1. Identificar columnas con problemas
        cols_with_missing = self.show_missing()
        if not cols_with_missing:
            return

        # 2. Crear Widgets
        col_dropdown = widgets.Dropdown(
            options=cols_with_missing,
            description='Columna a Limpiar:',
            style={'description_width': 'initial'}
        )

        # Opciones de imputación razonadas
        impute_options = [
            ('--- Selecciona ---', 'none'),
            ('Rellenar con Mediana (para datos numéricos sesgados)', 'median'),
            ('Rellenar con Media (para datos numéricos normales)', 'mean'),
            ('Rellenar con Moda (para datos categóricos)', 'mode'),
            ('Eliminar Filas (Usar con precaución)', 'dropna'),
            ('Imputación KNN (Avanzado, para numéricos)', 'knn')
        ]

        impute_dropdown = widgets.Dropdown(
            options=impute_options,
            description='Acción:',
            style={'description_width': 'initial'}
        )

        apply_button = widgets.Button(description="Aplicar y Actualizar")
        output_area = Output() # Aquí mostraremos los resultados

        def on_apply_button_clicked(b):
            col = col_dropdown.value
            action = impute_dropdown.value

            with output_area:
                clear_output()
                if action == 'none':
                    print("Por favor, selecciona una acción válida.")
                    return

                print(f"Aplicando '{action}' a la columna '{col}'...")

                # Guardamos el 'antes' para comparar
                missing_before = self.df_cleaned[col].isnull().sum()

                # --- Lógica de Limpieza ---
                if action == 'median':
                    if pd.api.types.is_numeric_dtype(self.df_cleaned[col]):
                        median_val = self.df_cleaned[col].median()
                        self.df_cleaned[col] = self.df_cleaned[col].fillna(median_val)
                        print(f"Rellenado con mediana: {median_val}")
                    else:
                        print(f"Error: '{col}' no es numérica. No se puede usar la mediana.")

                elif action == 'mean':
                    if pd.api.types.is_numeric_dtype(self.df_cleaned[col]):
                        mean_val = self.df_cleaned[col].mean()
                        self.df_cleaned[col] = self.df_cleaned[col].fillna(mean_val)
                        print(f"Rellenado con media: {mean_val}")
                    else:
                        print(f"Error: '{col}' no es numérica. No se puede usar la media.")

                elif action == 'mode':
                    mode_val = self.df_cleaned[col].mode().iloc[0]
                    self.df_cleaned[col] = self.df_cleaned[col].fillna(mode_val)
                    print(f"Rellenado con moda: {mode_val}")

                elif action == 'dropna':
                    rows_before = len(self.df_cleaned)
                    self.df_cleaned = self.df_cleaned.dropna(subset=[col])
                    rows_after = len(self.df_cleaned)
                    print(f"Se eliminaron {rows_before - rows_after} filas.")

                elif action == 'knn':
                    # (Esto es más complejo, requeriría seleccionar columnas numéricas)
                    print("Acción KNN (avanzada) aún no implementada en este esqueleto.")
                    # kNNImputer()...

                # --- Mostrar Resultados (Before/After) ---
                missing_after = self.df_cleaned[col].isnull().sum()
                print(f"Valores faltantes en '{col}' ANTES: {missing_before}")
                print(f"Valores faltantes en '{col}' DESPUÉS: {missing_after}")

                # Actualizar la lista de columnas en el dropdown
                new_cols_with_missing = self.show_missing()
                if new_cols_with_missing:
                    col_dropdown.options = new_cols_with_missing
                else:
                    col_dropdown.options = []
                    print("--- ¡Limpieza de faltantes completada! ---")


        apply_button.on_click(on_apply_button_clicked)

        # Mostrar todo
        display(VBox([
            HBox([col_dropdown, impute_dropdown]),
            apply_button,
            output_area
        ]))

In [ ]:
cleaner = InteractiveCleaner("data/titanic.csv", encoding='latin1')